In [32]:
import pandas as pd 

lab_a = pd.read_csv('Invitro.csv')
lab_b = pd.read_csv('OpenHealth.csv')
lab_c = pd.read_csv('Invivo.csv')

lab_a['Лаборатория'] = 'Invitro'
lab_b['Лаборатория'] = 'OpenHealth'
lab_c['Лаборатория'] = 'Invivo'

added the column with name of laboratory to dataframes

In [33]:
all_labs = pd.concat([lab_a, lab_b, lab_c], ignore_index=True)

united all labs into one dataframe

In [34]:
all_labs.columns

Index(['Код', 'Наименование', 'Биоматериал', 'Тип результата',
       'Лаборатория исполнения', 'Алматы, СРОК', 'Партнерская цена, в тенге.',
       'Цена для клиентов на сайте checkdoc', 'Лаборатория',
       'Перечень исследований', 'Розница', 'Партнерская', 'Тип контейнера',
       'Срок готовности', 'Цена', 'Цена со скидкой', 'Сроки выполнения'],
      dtype='object')

In [35]:
all_labs['Партнерская цена'] = all_labs['Партнерская'].combine_first(all_labs['Партнерская цена, в тенге.'])
all_labs['Наименование анализа'] = all_labs['Наименование'].combine_first(all_labs['Перечень исследований'])

all_labs.drop(columns='Партнерская', inplace=True)
all_labs.drop(columns='Партнерская цена, в тенге.', inplace=True)
all_labs.drop(columns='Наименование', inplace=True)
all_labs.drop(columns='Перечень исследований', inplace=True)


так как есть две разных колонки "партнерская" и "партнерская цена в тенге", которые хранят значения для разных лабораторий, объединила их в одну колонку - "партнерская цена". Она работает по принципу если есть значение из первой колонки, то оставляю его, если нет, то ставлю то, что во второй. Тоже самое с наименованием, а ненужные столбцы удаляем

In [36]:
all_labs["Наименование анализа"].value_counts()

Наименование анализа
Мочевина                                                                                                                                                                                                                                                              3
Витамин С (аскорбиновая кислота)                                                                                                                                                                                                                                      3
Тромбиновое время                                                                                                                                                                                                                                                     3
АПТВ (активированное парциальное тромбопластинновое время)                                                                                                                                 

In [37]:
def is_section_header(row):
    name = str(row['Наименование анализа'])
    return (
        name.isupper() and
        pd.isna(row['Код']) and
        pd.isna(row.get('Партнерская цена', None)) and
        pd.isna(row.get('Цена', None)) and
        pd.isna(row.get('Цена для клиентов на сайте checkdoc', None))
    )

all_labs['is_header'] = all_labs.apply(is_section_header, axis=1)


так как в изначальном документе в таблицах OpenHealth и Invivo классификация по категориям проводилась буквально перед всеми анализами (капс в наименовании), относящимся к этой категории, сейчас компьютер думает, что название категории это название еще одного анализа. То есть мы написали функцию, которая выбирает столбец Наименование, ищет название написаное капсом и при этом чтобы остальные строки были NaN. Так как мы указали axis=1, то он будет применять эту функцию к каждой строке и искать название секции. Apply() возвращает True, если всё что написано в теле функции окажется правдой 

In [38]:
category_col = []
current_category = None

for is_header, name in zip(all_labs['is_header'], all_labs['Наименование анализа']):
    if is_header:
        current_category = name.title() 
        category_col.append(None)  
    else:
        category_col.append(current_category)

all_labs['Категория'] = category_col

мы создаем пустой лист, куда будем складывать значения категорий для каждого из анализов (то есть построчно). Так же ставим значение текущей категории дефолтное - NaN. В цикле берем булевые значения (категория/не категория) и само наименование анализов. Если категория, то меняем значение текущей категории на это название (но уже не капсом), при этом в лист категорий добавляем NaN, потому что потом мы просто уберем эту строку (так как там лежит не наименование анализа и она нам не нужна). Но если это не категория, то тогда мы добавляем в список категорий название текущей категории, чтобы потом у этого анализа была именна эта категория. В конце уже создаем новую колонку категория, которой присваеваем значения списка

In [39]:
all_labs = all_labs[~all_labs['is_header']].copy()
all_labs.drop(columns='is_header', inplace=True)

`~all_labs['is_header']` тоже самое что НЕ КАТЕГОРИИ, то есть в датафрейме all_labs мы оставляем копию дата фрейма all_labs где в колонке "is_header" лежат не категории (то есть чистим от просто названий категорий)

а дальше уже просто удаляем и саму колонку "is_header", потому что она нам не нужна больше

In [40]:
all_labs['Партнерская цена'] = all_labs['Партнерская цена'].combine_first(all_labs['Цена со скидкой'])
all_labs.drop(columns="Цена со скидкой", inplace=True)

узнала, что партнерская цена = цена со скидкой по значению от ментора, так что теперь я добавила и это в парнерскую цену и удалила лищний столбец 

In [41]:
all_labs[all_labs['Партнерская цена'].isna() | (all_labs['Партнерская цена'].astype(str).str.strip() == '')]

,Код,Биоматериал,Тип результата,Лаборатория исполнения,"Алматы, СРОК",Цена для клиентов на сайте checkdoc,Лаборатория,Розница,Тип контейнера,Срок готовности,Цена,Сроки выполнения,Партнерская цена,Наименование анализа,Категория
1852,NaN,NaN,NaN,NaN,NaN,0.0,OpenHealth,NaN,NaN,NaN,NaN,NaN,NaN,ГЕМОСТАЗИОЛОГИЧЕСКИЕ ИССЛЕДОВАНИЯ,Гематологические Исследования
1870,NaN,NaN,NaN,NaN,NaN,0.0,OpenHealth,NaN,NaN,NaN,NaN,NaN,NaN,ОБЩЕКЛИНИЧЕCКИЕ ИCCЛЕДОВАНИЯ,Гематологические Исследования
1896,NaN,NaN,NaN,NaN,NaN,0.0,OpenHealth,NaN,NaN,NaN,NaN,NaN,NaN,БИОХИМИЧЕСКИЕ ИССЛЕДОВАНИЯ МОЧИ,Гематологические Исследования
1918,NaN,NaN,NaN,NaN,NaN,0.0,OpenHealth,NaN,NaN,NaN,NaN,NaN,NaN,БИОХИМИЧЕСКИЕ ИССЛЕДОВАНИЯ КРОВИ,Гематологические Исследования
1967,NaN,NaN,NaN,NaN,NaN,0.0,OpenHealth,NaN,NaN,NaN,NaN,NaN,NaN,Cпецифические белки (Specific proteins),Гематологические Исследования
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4148,NaN,NaN,NaN,NaN,NaN,NaN,Invivo,NaN,NaN,NaN,NaN,NaN,NaN,Антинуклеарный фактор на клеточной линии HEp-2...,Срб+ Рф+ Асло
4150,Исполнитель,NaN,NaN,NaN,NaN,NaN,Invivo,NaN,NaN,NaN,NaN,NaN,NaN,"Антинуклеарные антитела IgG (ANA: dsDNA, Nucle...",Срб+ Рф+ Асло
4153,NaN,NaN,NaN,NaN,NaN,NaN,Invivo,NaN,NaN,NaN,NaN,NaN,NaN,Антитела IgA и IgG к Saccharomyces cerevisiae ...,Срб+ Рф+ Асло
4155,NaN,NaN,NaN,NaN,NaN,NaN,Invivo,NaN,NaN,NaN,NaN,NaN,NaN,Антинуклеарный фактор на клеточной линии HEp-2...,Срб+ Рф+ Асло


Проверила, что названия категорий все еще остались типа как анализы. Так же в самом экселе клиники invivo оказались какие-то непонятные строчки, которые не несут в себе никакой реальной информации. Эти строчки не имеют никакой информации, там все колонки (кроме наименования и категории) это NaN.  